# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:


%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression

In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [3]:
df.describe()

,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables relative to whether someone got admitted or not. Think in terms of for a certain prestige level, how many people got admitted and didnt get admitted

In [4]:
# frequency table for prestige and whether or not someone was admitted
df.groupby('prestige').count()

,admit,gre,gpa
prestige,,,
1.0,61,61,61
2.0,148,148,148
3.0,121,121,121
4.0,67,67,67


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
#prestigedummy = pd.get_dummies(df.prestige, prefix='prestige')
prestigedummy = df.join(pd.get_dummies(df['prestige']))

In [6]:
prestigedummy.head()

,admit,gre,gpa,prestige,1.0,2.0,3.0,4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: One less than the total number of features

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [8]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestigedummy.ix[:, 'prestige':])
print handCalc.head()

   admit    gre   gpa  prestige  1.0  2.0  3.0  4.0
0      0  380.0  3.61       3.0    0    0    1    0
1      1  660.0  3.67       3.0    0    0    1    0
2      1  800.0  4.00       1.0    1    0    0    0
3      1  640.0  3.19       4.0    0    0    0    1
4      0  520.0  2.93       4.0    0    0    0    1


In [9]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [10]:
odds = 33.0/(28+33)
print odds

0.540983606557


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [16]:
print (df[(df.admit==1) & (df.prestige >1)].count()) / len(df)

#print df[df.admit>1].sum()
#print df([df.prestige > 1] & [df.admit==1]).sum()

admit       0.234257
gre         0.234257
gpa         0.234257
prestige    0.234257
dtype: float64


#### 3.3 Calculate the odds ratio

In [17]:
df[df.admit==1].count() / len(df)

admit       0.31738
gre         0.31738
gpa         0.31738
prestige    0.31738
dtype: float64

#### 3.4 Write this finding in a sentenance: 

Answer: Students have a 31.8% chance of gaining admission. If they went to a Prestige 1 School they have a 54.1% chance of admission. 

#### 3.5 Print the cross tab for prestige_4

In [22]:
pd.crosstab(handCalc.admit, handCalc.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


#### 3.6 Calculate the OR 

In [23]:
odds4 = 12.0/(55+12)
print odds4

0.179104477612


#### 3.7 Write this finding in a sentence

Answer: If the student attended a prestige 4 undergraduate university, they have a 17.9% of gaining addmistion

## Part 4. Analysis

In [25]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestigedummy.ix[:, 'prestige':])
print data.head()

   admit    gre   gpa  prestige  1.0  2.0  3.0  4.0
0      0  380.0  3.61       3.0    0    0    1    0
1      1  660.0  3.67       3.0    0    0    1    0
2      1  800.0  4.00       1.0    1    0    0    0
3      1  640.0  3.19       4.0    0    0    0    1
4      0  520.0  2.93       4.0    0    0    0    1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [26]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige,1.0,2.0,3.0,4.0,intercept
0,0,380.0,3.61,3.0,0,0,1,0,1.0
1,1,660.0,3.67,3.0,0,0,1,0,1.0
2,1,800.0,4.00,1.0,1,0,0,0,1.0
3,1,640.0,3.19,4.0,0,0,0,1,1.0
4,0,520.0,2.93,4.0,0,0,0,1,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [46]:
train_cols = data['prestige']

In [52]:
train_cols.head()

0    3.0
1    3.0
2    1.0
3    4.0
4    4.0
Name: prestige, dtype: float64

#### 4.2 Fit the model

In [47]:
lm2 = LogisticRegression()

In [53]:
lm2.fit(data[['gre', 'gpa', 1, 2, 3, 4]], df['admit'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### 4.3 Print the summary results

In [58]:
print lm2.coef_
print lm2.intercept_
print data.admit.mean()

[[ 0.00172658  0.19596439  0.35943456 -0.31929464 -0.88821548 -1.10459123]]
[-1.95266679]
0.317380352645


In [59]:
Score = data[['gre', 'gpa', 1, 2, 3,4]]
print lm2.score(Score, data['admit'])

0.712846347607


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [61]:
np.exp(lm2.coef_)

array([[ 1.00172808,  1.21648359,  1.43251918,  0.72666141,  0.41138923,
         0.3313463 ]])

In [69]:
lm3 = smf.ols(formula='admit ~ prestige', data=data).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admit   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     24.91
Date:                Sun, 07 May 2017   Prob (F-statistic):           9.02e-07
Time:                        13:12:57   Log-Likelihood:                -247.58
No. Observations:                 397   AIC:                             499.2
Df Residuals:                     395   BIC:                             507.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.6157      0.064      9.630      0.000         0.490     0.741
prestige      -0.1199      0.024     -4.991      0.000        -0.167    -0.073
==============================================================================
Omnibus:                      289.617   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.484
Skew:                           0.718   Prob(JB):                     2.00e-13
Kurtosis:                       1.785   Cond. No.                         8.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [66]:
lm3.conf_int()

,0,1
Intercept,0.489963,0.741347
prestige,-0.167064,-0.072643


#### 4.5 Interpret the OR of Prestige_2

0.3581081081081081

Answer: 0.06 coefficient for prestige_2 and admission. The model explains, roughly, 2% of the variance of the dataset

#### 4.6 Interpret the OR of GPA

In [64]:
train_cols1 = data[['gpa']]
lm = smf.ols(formula='admit ~ train_cols1', data=data).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admit   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     12.35
Date:                Sun, 07 May 2017   Prob (F-statistic):           0.000492
Time:                        13:07:12   Log-Likelihood:                -253.61
No. Observations:                 397   AIC:                             511.2
Df Residuals:                     395   BIC:                             519.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      -0.4066      0.207     -1.961      0.051        -0.814     0.001
train_cols1     0.2134      0.061      3.514      0.000         0.094     0.333
==============================================================================
Omnibus:                      564.082   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.088
Skew:                           0.749   Prob(JB):                     7.35e-15
Kurtosis:                       1.700   Cond. No.                         33.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Answer: Correlation coefficient is 0.21

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [20]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [21]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [22]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


In [86]:
# recreate the dummy variables
dummy = pd.get_dummies(combos, prefix='prestige')
# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

In [23]:
lm1 = smf.ols(formula, data=combos).fit()
lm1.summary()

NameError: name 'formula' is not defined

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.